In [1]:
!jt -t oceans16 -T -N
#!jt -l

In [2]:
nEvents = int(100/24*1e4)
nThread = 24
Ecm= 13.5

ids = [443, 445, 421, 411, -411]
N_part = len(ids)

nPSFs = ['nNNPDF30_nlo_as_0118_A2_Z1', 'nNNPDF10_nlo_as_0118_D2', 'TUJU21_nlo_2_1', 'nCTEQ15HIX_FullNuc_2_1'] #0-3
         #'nNNPDF30_nlo_as_0118_A2_Z1', 'nNNPDF20_nlo_as_0118_D2', 'nNNPDF10_nlo_as_0118_Au197', #4-6
         ##'EPPS16nlo_CT14nlo_Al27 ', 'EPPS16nlo_CT14nlo_Au197 ', '0 '] #7-9
opts = [12, 7]

In [3]:
import sys
cfg = open("/home/yoren/bnl/pythia8308/examples/Makefile.inc")
lib = "/home/yoren/bnl/pythia8308/lib"
for line in cfg:
    if line.startswith("PREFIX_LIB="): lib = line[11:-1]; break
sys.path.insert(0, lib)

options = ['SoftQCD:all=on', 'SoftQCD:inelastic=on', 'HardQCD:allwdi=on', 'HardQCD:all=on', #0-3
           'HardQCD:onlydi=on', 'PromptPhoton:all=on', 'PhaseSpace:Q2Min=2.', 'PDF:pSet=LHAPDF6:cteq6l1',#4-7
           'MultipartonInteractions:Kfactor=0.5', 'PromptPhoton:all=on', 'PromptPhoton:qg2qgamma=on',#8-10
          ' PromptPhoton:qqbar2ggamma=on','Charmonium:all=on']#11-12

pdf_name = ['nNNPDF30', 'nNNPDF10', 'TUJU21', 'nCTEQ15',
            'nNNPDF30_nlo_as_0118_A2_Z1', 'nNNPDF20_nlo_as_0118_D2', 'nNNPDF10_nlo_as_0118_Au197',
            'EPPS16nlo_CT14nlo_Al27', 'EPPS16nlo_CT14nlo_Au197', '']

#!source /home/yoren/bnl/pythia8308/root_v6.26.10.Linux-ubuntu20-x86_64-gcc9.4.tar.gz/root/bin/thisroot.sh
! which root
import ROOT
#ROOT.gROOT.SetBatch()
#ROOT.ROOT.EnableThreadSafety()
#nthereds = 5
#fileName = "NICA/test.root"
#merger = ROOT.ROOT.TBufferMerger(fileName)
%jsroot on

/home/yoren/bnl/pythia8308/root6/bin/root
Welcome to JupyROOT 6.26/10


In [4]:
import lhapdf
lha_npdf_vec=[]
lha_npdf_set_vec=[]

for nPDF in nPSFs:
        lha_npdf_vec.append(lhapdf.mkPDFs(nPDF))
        lha_npdf_set_vec.append(lhapdf.getPDFSet(nPDF))
        
def_npdf = lhapdf.mkPDFs("cteq6l1")

LHAPDF 6.5.3 loading all 201 PDFs in set nNNPDF30_nlo_as_0118_A2_Z1
nNNPDF30_nlo_as_0118_A2_Z1, version 1; 201 PDF members
LHAPDF 6.5.3 loading all 251 PDFs in set nNNPDF10_nlo_as_0118_D2
nNNPDF10_nlo_as_0118_D2, version 1; 251 PDF members
LHAPDF 6.5.3 loading all 59 PDFs in set TUJU21_nlo_2_1
TUJU21_nlo_2_1, version 1; 59 PDF members
LHAPDF 6.5.3 loading all 43 PDFs in set nCTEQ15HIX_FullNuc_2_1
nCTEQ15HIX_FullNuc_2_1, version 2; 43 PDF members
LHAPDF 6.5.3 loading all 1 PDFs in set cteq6l1
cteq6l1, version 4; 1 PDF members


In [5]:
def weights_PDF(result_w, lha_npdf, lha_npdf_set, pdfid, pdfx, pdfscale, pdfpdf):

    pdf_vec = []
    for pdf in lha_npdf:
        pdf_vec.append(pdf.xfxQ2(pdfid, pdfx, pdfscale))
    
    #auto xgAll = lha_npdf[0]->xfxQ2(pdfid1, pdfx1, pdfscale);
    xgErr = lha_npdf_set.uncertainty(pdf_vec)

    pdf2_c = xgErr.central / pdfx
    pdf2_up = (xgErr.central + xgErr.errplus) / pdfx
    pdf2_down = (xgErr.central - xgErr.errminus) / pdfx
    #print(pdfx,lha_npdf[0].xfxQ2(pdfid, pdfx, pdfscale))
    pdf_weight_c = 1.0
    pdf_weight_up = 1.0
    pdf_weight_down = 1.0
    
    if pdfpdf>0.000001 and pdf2_c>0.000001 and pdfscale>=0.0:
        pdf_weight_c =  pdf2_c / pdfpdf
        pdf_weight_up = pdf2_up / pdfpdf
        pdf_weight_down = pdf2_down / pdfpdf
    else:
        a = 0
        #pdf_weight_c = 0;
        #pdf_weight_up = 0;
        #pdf_weight_down = 0;
    
    result_w.append(pdf_weight_c)
    result_w.append(pdf_weight_up)
    result_w.append(pdf_weight_down)
    

In [6]:
# Import the Pythia module.
import pythia8

N_sets = len(lha_npdf_vec)
N_bin_hist= 1 + 3*N_sets

def pythia8_runner(seed, nEvents_tred, target =2212, projectile=2212):
    #print("YOLO"*100,seed,projectile,target)
    
    hist_Nch = ROOT.TH1D("hist_Nch","hist_Nch",100,0,100)
    hist_xfx = ROOT.TH3D("hist_xfx","hist_xfx",100,0,1,100,0,3,100,0,Ecm)
    hist_part = ROOT.TH3D("hist_part","hist_part",20,0,10,N_bin_hist,0,N_bin_hist,10,0,10)
    hist_id = ROOT.TH2D("hist_id","id",25,-3,22,3,0,3)
    hist_sigma = ROOT.TH2D("hist_sigma","hist_sigma",100,0,10,3,0,3)

    pythia = pythia8.Pythia()
    pythia.readString(f"Beams:eCM = {Ecm}")
    for iopt in opts:
        pythia.readString(options[iopt])    
    pythia.readString("Random:setSeed = on")
    pythia.readString("421:mayDecay = off")
    pythia.readString("443:mayDecay = off")
    pythia.readString("411:mayDecay = off")
    pythia.readString("445:mayDecay = off")
    pythia.readString("PartonLevel:MPI = off")
    #pythia.readString("PartonLevel:ISR = off")
    #pythia.readString("PartonLevel:FSR = off")
    pythia.readString(f"Random:seed = {seed}")
    pythia.readString(f"Beams:idA = {projectile}") # The coper ion
    pythia.readString(f"Beams:idB = {target}") # The lead ions.
    pythia.init()
        # Begin event loop. Generate event. Skip if error. List first one.
    for iEvent in range(0, int(nEvents_tred)):
        #print(iEvent, seed)
        if not pythia.next(): continue
        
        info = pythia.infoPython()
        weight_pythia_pp = info.sigmaGen()*10000 
        weight_pythia = weight_pythia_pp #info.weight() 
        hist_id.Fill(info.id1pdf(),(projectile-2112.)/100+(target-2112.)/100)
        #if seed==15156: print(15156,projectile,target,info.id1pdf(),info.id2pdf(),weight_pythia)
                
        pdfid1 = info.id1pdf()
        pdfid2 = info.id2pdf()

        hist_sigma.Fill(weight_pythia,(projectile-2112.)/100+(target-2112.)/100)
            
        if projectile == 2112:
            pdfid10 = pdfid1
            if pdfid1 == 1:  pdfid10 = 2
            if pdfid1 == 2:  pdfid10 = 1
            if pdfid1 == -1: pdfid10 = -2
            if pdfid1 == -2: pdfid10 = -1
            pdfid1 = pdfid10
        if target == 2112:
            pdfid20 = pdfid2
            if pdfid2 == 1:  pdfid20 = 2
            if pdfid2 == 2:  pdfid20 = 1
            if pdfid2 == -1: pdfid20 = -2
            if pdfid2 == -2: pdfid20 = -1
            pdfid2 = pdfid20
            
        pdfx1 = info.x1pdf()
        pdfx2 = info.x2pdf()
        pdfscale = info.Q2Fac()
        pdfpdf1 = info.pdf1()
        pdfpdf2 = info.pdf2()

        if pdfscale<0.0: continue

        xfxQ2 = lha_npdf_vec[0][0].xfxQ2(pdfid1, pdfx1, pdfscale)
        hist_xfx.Fill(pdfx1,xfxQ2,pdfscale,weight_pythia)
        
        pdfpdf1 = def_npdf[0].xfxQ2(info.id1pdf(), pdfx1, pdfscale) / pdfx1
        pdfpdf2 = def_npdf[0].xfxQ2(info.id2pdf(), pdfx2, pdfscale) / pdfx2
        
        #pdfpdf1 = def_npdf[0].xfxQ2(pdfid1, pdfx1, pdfscale) / pdfx1
        #pdfpdf2 = def_npdf[0].xfxQ2(pdfid2, pdfx2, pdfscale) / pdfx2
        
        result_projec = [1.0]
        result_target = [1.0]
        for i in range(len(lha_npdf_vec)):
            weights_PDF(result_projec,lha_npdf_vec[i],lha_npdf_set_vec[i],pdfid1,pdfx1,pdfscale,pdfpdf1)
            weights_PDF(result_target,lha_npdf_vec[i],lha_npdf_set_vec[i],pdfid2,pdfx2,pdfscale,pdfpdf2)
        result = result_projec
        for i in range(len(result_projec)):
            result[i]=result_projec[i]*result_target[i]
        #print(result,result_projec,result_target)
        #weight_pythia = 1
        # Find number of all final charged particles and fill histogram.
        nCharged = 0
        for pythia_event in pythia.event:
            if abs(pythia_event.eta()) < 0.5 and pythia_event.isFinal():
                for iid in range(N_part):
                    if pythia_event.id() == ids[iid]:
                        for i in range(N_bin_hist):
                            hist_part.Fill(pythia_event.pT(), i, iid, weight_pythia_pp*result[i])
                            hist_part.Fill(pythia_event.pT(), i, iid+5, weight_pythia)
                            #if i==0:print("2rkk",iEvent, pythia_event.id(), mother_id, grandmother_id, pythia_event.pT(), pdfscale)


            if pythia_event.isFinal() and pythia_event.isCharged():
                nCharged += 1
        hist_Nch.Fill(nCharged,weight_pythia)
    
        
# End of event loop. Statistics. Histogram. Done.
    pythia.stat()
    return [hist_Nch, hist_xfx, hist_part, hist_id, hist_sigma]

In [7]:
#second(156,5000)
import multiprocess as mp

Ntr = nThread
Nev = nEvents
seed0 = 15156
seed_iter = 43324

pn_ids = [int(2212), int(2112)]

projectile_params = [pn_ids[int(1.34*i/(Ntr))] for i in range(Ntr)]
target_params = [pn_ids[min(int(4*i/Ntr), 1)] for i in range(Ntr)]
print(projectile_params, target_params)

pool = mp.Pool(Ntr)
#hist_array = pool.starmap(pythia8_runner, [(seed0+i*seed_iter, Nev, 2212, 2212) for i in range(Ntr)])
hist_array = pool.starmap(pythia8_runner, [(seed0+i*seed_iter, Nev, projectile_params[i], target_params[i]) for i in range(Ntr)])
pool.close()

[2212, 2212, 2212, 2212, 2212, 2212, 2212, 2212, 2212, 2212, 2212, 2212, 2212, 2212, 2212, 2212, 2212, 2212, 2112, 2112, 2112, 2112, 2112, 2112] [2212, 2212, 2212, 2212, 2212, 2212, 2112, 2112, 2112, 2112, 2112, 2112, 2112, 2112, 2112, 2112, 2112, 2112, 2112, 2112, 2112, 2112, 2112, 2112]

 *------------------------------------------------------------------------------------* 
 |                                                                                    | 
 |  *------------------------------------------------------------------------------*  | 
 |  |                                                                              |  | 
 |  |                                                                              |  | 
 |  |   PPP   Y   Y  TTTTT  H   H  III    A      Welcome to the Lund Monte Carlo!  |  | 
 |  |   P  P   Y Y     T    H   H   I    A A     This is PYTHIA version 8.308      |  | 
 |  |   PPP     Y      T    HHHHH   I   AAAAA    Last date of change: 16 Nov 2022  |  

In [8]:
hist_Nch = hist_array[0][0]
for i in range(1, Ntr):
    hist_Nch.Add(hist_array[i][0])
hist_xfx = hist_array[0][1]
for i in range(1,Ntr):
    hist_xfx.Add(hist_array[i][1]) 
hist_pi0 = hist_array[0][2]
for i in range(1, Ntr):
    hist_pi0.Add(hist_array[i][2])
hist_pi0_ref = hist_array[int(nThread/4-1)][2]
for i in range(int(nThread/4-2), 0,-1):
    hist_pi0_ref.Add(hist_array[i][2])
hist_id = hist_array[0][3]
for i in range(1, Ntr):
    hist_id.Add(hist_array[i][3])
hist_sigma = hist_array[0][4]
for i in range(1, Ntr):
    hist_sigma.Add(hist_array[i][4])

In [9]:
c5 = ROOT.TCanvas("c05","c05",1080,360)
c5.Divide(3,1)
sigmas=[]
for i in range(3):
    c5.cd(i+1)
    hist_sigma_NN = hist_sigma.ProjectionX(f"hist_sigma_proj{i}",i+1,i+1)
    hist_sigma_NN.SetLineColor(4)
    hist_sigma_NN.Draw()
    sigmas.append(hist_sigma_NN.GetEntries()*hist_sigma_NN.GetMean())
reweight = sum(sigmas)/4/sigmas[2]
print(reweight)
c5.Print("NICA/sigmas.png")
c5.Draw()

1.0006386064040227


Info in <TCanvas::Print>: png file NICA/sigmas.png has been created


In [10]:
ids_new = [443, 445, 421, 411, -411, 1443, 1445, 1421, 1411, -1411]
N_part_new = len(ids_new) 
hists_to_write = []
canvases =[]
for ipart in range(N_part_new):
    
    c2 = ROOT.TCanvas(f"c{ipart}",f"c{ipart}",1080,1080)
    c2.Divide(2,2)
    print(ipart)
#print(len(hist_array),N_bin_hist,len(nPSFs),len(hist_array[0]),hist_pi0.GetNbinsY())
    part_type = ipart+1
    part_ref=part_type
    for i in range(N_sets):
        c2.cd(i+1)
        hist_referen = hist_pi0.ProjectionX(f"hist_pi0_proj_ref{i}_{ids_new[ipart]}",1,1,part_ref,part_ref)
        if ipart>4:
            hist_referen = hist_pi0_ref.ProjectionX(f"hist_pi0_proj_ref{i}_{ids_new[ipart]}",1,1,part_ref,part_ref)
            hist_referen.Scale(4.)
        #hist_referen.Scale(1./reweight)
        hist_uppredg = hist_pi0.ProjectionX(f"hist_upperedge{i}_{ids_new[ipart]}",3+i*3,3+i*3,part_type,part_type)
        hist_uppredg.Divide(hist_referen)
        hist_uppredg.SetLineColor(6)
        hist_uppredg.Draw()
        hist_central = hist_pi0.ProjectionX(f"hist_central{i}_{ids_new[ipart]}",2+i*3,2+i*3,part_type,part_type)
        hist_central.Divide(hist_referen)
        hist_central.SetLineColor(4)
        hist_central.Draw("same")
        hist_lowedge = hist_pi0.ProjectionX(f"hist_lowedge{i}_{ids_new[ipart]}",4+i*3,4+i*3,part_type,part_type)
        hist_lowedge.Divide(hist_referen)
        hist_lowedge.SetLineColor(2)
        hist_lowedge.Draw("same")
        hists_to_write.append(hist_central)
        hists_to_write.append(hist_lowedge)
        hists_to_write.append(hist_uppredg)
    
    #c2.Print(f"NICA/NICA{ids_new[ipart]}_{int(Nev*nThread)}.pdf")
    c2.Print(f"NICA/NICA{ids_new[ipart]}_{int(Nev*nThread)}.png")
    canvases.append(c2)

0
1
2
3
4
5
6
7
8
9


Info in <TCanvas::Print>: png file NICA/NICA443_999984.png has been created
Info in <TCanvas::Print>: png file NICA/NICA445_999984.png has been created
Info in <TCanvas::Print>: png file NICA/NICA421_999984.png has been created
Info in <TCanvas::Print>: png file NICA/NICA411_999984.png has been created
Info in <TCanvas::Print>: png file NICA/NICA-411_999984.png has been created
Info in <TCanvas::Print>: png file NICA/NICA1443_999984.png has been created
Info in <TCanvas::Print>: png file NICA/NICA1445_999984.png has been created
Info in <TCanvas::Print>: png file NICA/NICA1421_999984.png has been created
Info in <TCanvas::Print>: png file NICA/NICA1411_999984.png has been created
Info in <TCanvas::Print>: png file NICA/NICA-1411_999984.png has been created


In [11]:
canvases[0].Draw()     # [443, 445, 421, 411, -411]

In [12]:
c4 = ROOT.TCanvas("c04","c04",1080,360)
c4.Divide(3,1)
for i in range(3):
    c4.cd(i+1)
    hist_id_NN = hist_id.ProjectionX(f"hist_sigma_proj{i}",i+1,i+1)
    hist_id_NN.SetLineColor(4)
    hist_id_NN.Draw()
   
c4.Print("NICA/id.png")
c4.Draw()

Info in <TCanvas::Print>: png file NICA/id.png has been created


In [13]:
stop

NameError: name 'stop' is not defined

In [ ]:
output_file = ROOT.TFile.Open(f"NICA/dd_outputfile_{N_sets}npdf_{int(Nev*nThread)}_at_{int(Ecm)}.root","RECREATE")
hist_pi0.Write()
for hist in hists_to_write:
    hist.Write()
output_file.Close()

In [ ]:
import numpy as np 

Nx = hists_to_write[0].GetNbinsX()+1
pt = [hists_to_write[0].GetBinCenter(i) for i in range(1,Nx)]
data = [[hists_to_write[j].GetBinContent(i) for i in range(1,Nx)] 
               for j in range((N_bin_hist-1)*N_part)]
data.insert(0,pt)
print(data)
np_data = np.array(data)

np.save(f"NICA/data_{N_sets}npdf_{int(Nev*nThread)}_at_{int(Ecm)}.npy", np_data)

data_array = np.load(f"NICA/data_{N_sets}npdf_{int(Nev*nThread)}_at_{int(Ecm)}.npy")
print(data_array)


In [ ]:
import json

def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

names=['central', 'upper', 'lower']

print(hists_to_write[0].GetBinContent(1),hists_to_write[3].GetBinContent(1))

data_dictionary = {f'{names[j%3]}_{pdf_name[int(j/3)%N_sets]}_{ids_new[int(j/3/N_sets)%N_part_new]}': [hists_to_write[j].GetBinContent(i) for i in range(1,Nx)]
               for j in range(len(hists_to_write))}
details = {'pt': pt}
fulldata_dictionary = Merge(details,data_dictionary)

with open(f"NICA/data_{N_sets}npdf_{int(Nev*nThread)}_at_{int(Ecm)}.txt", 'w') as convert_file:
    convert_file.write(json.dumps(fulldata_dictionary))

In [ ]:
with open(f"NICA/data_{N_sets}npdf_{int(Nev*nThread)}_at_{int(Ecm)}.txt") as f:
    data = f.read()
    
dictionary = json.loads(data)

print("Data type after reconstruction : ", type(dictionary))
print(dictionary['pt'],dictionary['central_nNNPDF10_22'],dictionary['central_TUJU21_22']) #'nNNPDF30', 'nNNPDF10', 'TUJU21', 'nCTEQ15',

In [ ]:
c0 = ROOT.TCanvas("c0","c0",720,720)

part_type = 2

c0.Divide(2,2)
c0.cd(1)
hist_pi0.Draw()
c0.cd(2)
hist_xfx.Draw()
c0.cd(3)
hist_pi0_proj = hist_xfx.ProjectionZ("hist_pi0_proj",1,-1,1,-1)
ax = hist_pi0_proj.GetXaxis()
hist_pi0_proj.Draw()
c0.cd(4)
c0.Draw()
%jsroot on
